In [7]:
import xgboost
import shap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf

df = pd.read_csv("C:\\Users\\Nicol\\Desktop\\bike_rental.csv")

In [8]:
X = df.drop(columns=['cnt'])
y = df['cnt']

In [9]:
np.random.seed(42)
tf.random.set_seed(42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Gradient Boosting
model1 = xgboost.XGBRegressor(random_state=42).fit(X,y)

# NN
model2 = Sequential()
model2.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model2.add(Dense(1))
model2.compile(loss='mse', optimizer='adam')
model2.fit(X_train, y_train, epochs=50)

Epoch 1/50
435/435 [==============================] - 1s 1ms/step - loss: 30892.7969
Epoch 2/50
435/435 [==============================] - 1s 2ms/step - loss: 28918.2461
Epoch 3/50
435/435 [==============================] - 1s 2ms/step - loss: 27546.4590
Epoch 4/50
435/435 [==============================] - 1s 2ms/step - loss: 26276.6191
Epoch 5/50
435/435 [==============================] - 1s 2ms/step - loss: 25426.0332
Epoch 6/50
435/435 [==============================] - 1s 3ms/step - loss: 24865.4590
Epoch 7/50
435/435 [==============================] - 1s 3ms/step - loss: 24795.1621
Epoch 8/50
435/435 [==============================] - 1s 3ms/step - loss: 24574.5859
Epoch 9/50
435/435 [==============================] - 1s 3ms/step - loss: 24308.7773
Epoch 10/50
435/435 [==============================] - 1s 1ms/step - loss: 24297.5176
Epoch 11/50
435/435 [==============================] - 1s 1ms/step - loss: 24100.0059
Epoch 12/50
435/435 [==============================] - 1s 1ms/s

In [11]:
# Predicting results

pred1 = model1.predict(X)
preds2 = model2.predict(X)

explainer1 = shap.Explainer(model1)
explainer2 = shap.DeepExplainer(model2, X.values)

shap_values1 = explainer1(X)
shap_values2 = explainer2.shap_values(X.values)


You have provided over 5k background samples! For better performance consider using smaller random sample.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


In [13]:
with open("XGB_vs_NN.txt", "w") as f:
    for i in range(0, len(y)):
        f.write(str(pred1[i]))
        f.write(", ")
        f.write(str(preds2[i]))
        f.write("\n")

In [ ]:
max_y = max(y)
for i in range(0, len(y)):
    pred1[i] = pred1[i] / max_y
    preds2[i] = preds2[i] / max_y

with open("two_models_preds_shaps.txt", "w") as f:
    for i in range(0, len(y)):
        f.write(str(pred1[i]))
        f.write(", ")
        f.write(str(preds2[i]))
        f.write(", ")
        tmp_lst1 = []
        tmp_lst2 = []
        for j in range(0, len(shap_values1[i].values)):
            tmp_lst1.append(str(shap_values1[i].values[j]))
            tmp_lst1.append(", ")
        tmp_strng1 = ''.join(tmp_lst1)
        f.write(tmp_strng1)
        tmp_lst2 = []
        for j in range(0, len(shap_values2[0][i])):
            tmp_lst2.append(str(shap_values2[0][i][j]))
            tmp_lst2.append(", ")
        tmp_strng2 = ''.join(tmp_lst2)
        nw_ts2 = tmp_strng2[:-2]
        f.write(nw_ts2)
        f.write("\n")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [ ]:
N = len(X)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(X)

In [ ]:
shap_diffs = []
for i in range(0, len(y)):
    cur_shaps = []
    for j in range(0,len(shap_values1[i].values)):
        cur_d = shap_values1[i].values[j] - shap_values2[0][i][j]
        cur_shaps.append(cur_d)
    shap_diffs.append(cur_shaps)

In [ ]:
df = pd.DataFrame(shap_diffs)
feature_names = list(X.columns)
df.columns = feature_names

In [ ]:
# Load libraries and modules
import pandas as pd
from sklearn import preprocessing
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.linear_model import LinearRegression
from sklearn.metrics.cluster import normalized_mutual_info_score
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
from sklearn.metrics import silhouette_score

# Extract the features into X
X1 = df.iloc[:, 1:].values

# Calculate the silhouette score for different numbers of clusters
scores = []
for n_clusters in range(2, 20):
    clusterer = KMeans(n_clusters=n_clusters)
    cluster_labels = clusterer.fit_predict(X1)
    silhouette_avg = silhouette_score(X1, cluster_labels)
    scores.append(silhouette_avg)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

# Find the optimal number of clusters
optimal_n_clusters = np.argmax(scores) + 2
print("Optimal number of clusters =", optimal_n_clusters)


In [ ]:
kmeans = KMeans(n_clusters=optimal_n_clusters)
kmeans.fit(df)
labels=kmeans.labels_
df['cluster'] = labels
print(kmeans.cluster_centers_)

In [ ]:
# t-SNE and scatterplot on original dataset, coloring it basing on cluster beloning

from sklearn.manifold import TSNE
import seaborn as sns

tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)

df2 = pd.DataFrame(X_tsne, columns=['tsne1','tsne2'])
df2['label'] = y
df2['cluster'] = df['cluster']

sns.scatterplot(data=df2, x='tsne1', y='tsne2', hue='cluster', palette='bright')


In [ ]:
counts = df2['cluster'].value_counts()
print(counts)